# TP 26/03

In [1]:
import tkinter.font as tkFont
import time
try:
    import tkinter as tk
    from tkinter import *
except:
    import Tkinter as tk
    from Tkinter import *

In [2]:
class Game():
    def __init__(self, frame):
        self.frame = frame
        self.mise_en_page()
        self.fleet = Fleet()
        self.defender = Defender()
        self.height =600
        self.width =600
        self.canvas = tk.Canvas(self.frame,width=self.width, height=self.height, background='pink' )
        self.canvas.pack()
        self.score=0
        self.tag="score"
        self.affichage_score()
        self.defender.install_in(self.canvas)
        self.fleet.install_in(self.canvas)
        self.gap=-20
        self.feu=None
        self.cpt=0
        self.fini=0
        self.y=0
        self.nb_y=0
        self.dx=5
        self.texte=None
        
        
    def mise_en_page(self):
        #création d'un esapce à l'écriture pour les règles du jeu
        labelframe=self.frame
        labelframe.pack(side=LEFT)
        police=tkFont.Font(family='Trebuchet MS', size=14)
        label=Label(self.frame,text="Bienvenue !\nCe jeu est simple, tu es Luci et pour ne\n pas mourir et avoir un super score,\n tu dois tuer les supers monstres en leur envoyant des boules de feu.\n Mais ce ne sera pas si simple,\n car il faudra élilminer toutes les carapaces qui les\n protègent en leur envoyant une boule de feu,\n ce qui te ferras gagner 200 points.\nPour éliminer un super monstre,\n tu devras leurs envoyer 4 boules de feu en même temps,\n ce qui te fera gagner 500 points.\nOublie pas d'appuyer sur 'JOUER' pour que le jeu commence!\n Bonne chance !",background='#9EFF79', font=police)
        label.pack(side=LEFT)
        #création du bouton 'JOUER'
        self.Gamebutton = Button(self.frame, text="JOUER", fg="white",width=20, font=police,background='yellow',command=lambda: self.start_animation(),activebackground='red')
        self.Gamebutton.pack(side=BOTTOM)
        self.Gamebutton.flash()
        
    #lance la fonction animation et supprime le bouton 'JOUER'
    def start_animation(self):
        self.canvas.after(10, self.animation)
        self.Gamebutton.pack_forget()
     
    #fonction récursive jusqu'à ce que self.fini=1 dans move_aliens_fleet
    def animation(self):
        self.move_bullets()
        self.move_aliens_fleet()
        self.score=self.fleet.manage_touched_aliens_by(self.canvas,self.score)
        self.affichage_score()
        self.score=self.fleet.manage_touched_mega_aliens_by(self.canvas,self.score)
        self.affichage_score()
        if self.fini==0:
            self.canvas.after(100,self.animation)
            
    #lancer du bullet et la supprime si elle sort de la fenêtre
    def move_bullets(self):
        for bullet in self.defender.fired_bullets.copy():
            x1,y1=self.canvas.coords(bullet.id)
            bullet.move_in(self.canvas)
            if y1<0:
                self.canvas.delete(bullet.id)
                self.defender.fired_bullets.remove(bullet)
                
    #gère le déplacement de la fleet dans un premier temps, puis détecte si le joueur a gagné ou perdu          
    def move_aliens_fleet(self):
        police = tkFont.Font(family='Helvetica', size=40)
        police1 = tkFont.Font(family='Helvetica', size=20)
        if self.nb_y<19 and self.fleet.nb_alien_mort!=(self.fleet.fleet_size+self.fleet.nb_mega_alien):
            if self.cpt<6:
                self.fleet.move_in(self.canvas,self.dx,self.y)
                self.cpt=self.cpt+1
                self.y=0
            if self.cpt==6:
                self.dx=-self.dx
                self.cpt=-5
                self.y=10
                self.nb_y=self.nb_y+1
        elif self.nb_y==19:
            TEXTE="TU AS PERDU!"
            self.canvas.create_text(self.width//2,self.height//2,font=police,text=TEXTE)
            self.canvas.create_text(self.width//2,self.height//2+60,font=police1,text="Ton score final est de "+str(self.score))
            self.fini=1
        elif self.fleet.nb_alien_mort==(self.fleet.fleet_size+self.fleet.nb_mega_alien):
            TEXTE="TU AS GAGNÉ!"
            self.canvas.create_text(self.width//2,self.height//2,font=police,text=TEXTE)
            self.canvas.create_text(self.width//2,self.height//2+60,font=police1,text="Ton score final est de "+str(self.score))
            self.fini=1
            
    #lors d'un événement du clavier, appel la fonction associée
    def keypress(self, event):
        dx,dy=0,0
        if event.keysym == 'Left':
            self.defender.move_in(self.canvas,-10)
        elif event.keysym == 'Right':
            self.defender.move_in(self.canvas,10)
        elif event.keysym == 'space':
            self.defender.fire(self.canvas)
            
    #affiche le score via un tag
    def affichage_score(self):
        if self.score !=0:
            self.canvas.delete(self.tag)
        TEXTE="score: "+str(self.score)
        police = tkFont.Font(family='Helvetica', size=20)
        self.canvas.create_text(self.width//2,20,font=police,text=TEXTE,tags=self.tag)
            

In [3]:
class Defender():
    def __init__(self):
        self.width = 119
        self.height = 41
        self.move_delta = 20
        self.id = None
        self.max_fired_bullets= 8
        self.fired_bullets = []
        self.image_lucie=tk.PhotoImage(file='lucie.png')
        
    #fait la mise en place de Luci
    def install_in(self, canvas):
        x0=int(canvas['width'])//2-(self.width//2)
        y0=int(canvas['height'])-self.height
        x1=x0+self.width
        y1=y0+self.height+1
        self.id=canvas.create_image(x0,y0,image=self.image_lucie)
    
    #permet le déplacement de Luci a droite et à gauche délimité par la fenêtre
    def move_in(self,canvas,dx):
        if(canvas.coords(self.id)[0]<int(canvas['width'])-(self.width//2) and dx>0):
            canvas.move(self.id,dx,0)
        if(canvas.coords(self.id)[0]>(self.width//2)+10 and dx<0):
            canvas.move(self.id,dx,0)

    #créer de nouvelles boules de feu 
    def fire(self, canvas):
        if len(self.fired_bullets)<self.max_fired_bullets:
            newBullet=Bullet(self)
            newBullet.install_in(canvas)
            self.fired_bullets.append(newBullet)

In [4]:
class Bullet():
    def __init__(self, shooter):
        self.radius = 8
        self.speed = 20
        self.id = None
        self.shooter = shooter
        self.image_feu=tk.PhotoImage(file='balle_enflammee.png')
        self.width=12
        self.heigth=34
        
    #fait la mise en place de la boule de feu
    def install_in(self, canvas):
        x1=canvas.coords(self.shooter.id)[0]
        y1=canvas.coords(self.shooter.id)[1]
        x0=x1+(2*self.radius-self.radius//2)
        y0=y1-3*self.radius
        self.id=canvas.create_image(x0,y0,image=self.image_feu)
 
    #déplacement de la boule de feu
    def move_in(self, canvas):
        canvas.move(self.id,0,-self.speed)

In [5]:
class Fleet(object):
    def __init__(self):
        self.aliens_lines = 2
        self.aliens_columns = 10
        self.alien_x_delta = 7
        self.alien_y_delta = 7 
        self.fleet_size = self.aliens_lines * self.aliens_columns
        self.aliens_fleet = [None] * self.fleet_size
        self.alien_width=48
        self.alien_height=40
        self.nb_mega_alien=3
        self.mega_aliens_fleet=[None]*self.nb_mega_alien
        self.mega_alien_width=152
        self.mega_alien_height=174
        self.mega_alien_delta=35
        self.num_mega_alien=0
        self.cpt=0
        self.nb_alien_mort=0
        
    #mise en place des grands aliens avec la class Mega_alien() et des carapaces avec Alien()
    def install_in(self, canvas):
        x=100
        y=125
        for j in range(len(self.mega_aliens_fleet)):
            self.mega_aliens_fleet[j]=Mega_alien()
            self.mega_aliens_fleet[j].install_in(canvas,x,y,self.num_mega_alien)
            x=x+self.mega_alien_width+self.mega_alien_delta
            self.num_mega_alien=self.num_mega_alien+1
        x=50
        y=250
        for i in range(len(self.aliens_fleet)):
            self.aliens_fleet[i]=Alien()
            self.aliens_fleet[i].install_in(canvas,x,y)
            x=x+self.alien_width+self.alien_x_delta
            if x>(int(canvas['width'])-self.alien_width):
                x=50
                y=y+self.alien_height+self.alien_y_delta

    #mouvement fleet de carapaces seulement    
    def move_in(self, canvas,x,y):
        for alien in self.aliens_fleet:
            alien.move_in(canvas,x,y)
            
    #détecte s'il y a une collision, appel la fonction qui supprime un méga alien et retourne le score
    def manage_touched_mega_aliens_by(self,canvas,score):
        for mega_alien in self.mega_aliens_fleet.copy():
            if mega_alien.id != [None]:
                x1,y1,x2,y2=canvas.bbox(mega_alien.id)
                if len(canvas.find_overlapping(x1,y1,x2,y2))>4:
                    score=int(score)+500
                    mega_alien.touched_by(canvas)
                    mega_alien.id=[None]
                    self.nb_alien_mort=self.nb_alien_mort+1
        return score
    
    #détecte s'il y a une collision, appel la fonction qui supprime un alien et retourne le score
    def manage_touched_aliens_by(self,canvas,score):
        for alien in self.aliens_fleet.copy():
            if alien.id!=[None]:
                x1,y1,x2,y2=canvas.bbox(alien.id)
                if len(canvas.find_overlapping(x1,y1,x2,y2))>2:
                    score=int(score)+200
                    alien.touched_by(canvas)
                    alien.id=[None]
                    self.nb_alien_mort=self.nb_alien_mort+1
        return score         

In [6]:
class Alien():
    def __init__(self):
        self.id = None
        self.alive = True       
        self.image_carapace=tk.PhotoImage(file='carapace.gif')

    #supprime une carapace
    def touched_by(self, canvas):
        canvas.delete(self.id)
    
    #créer l'image de la carapace
    def install_in(self, canvas, x, y):
        img=self.image_carapace
        self.id=canvas.create_image(x,y,image=img)
        
    #mouvement d'un alien
    def move_in(self, canvas, dx, dy):
        canvas.move(self.id,dx,dy)

In [7]:
class Mega_alien():
    def __init__(self):
        self.id = None
        self.alive = True       
        self.image_King=tk.PhotoImage(file='kingkong.png')
        self.image_Crocodile=tk.PhotoImage(file='crocodile.png')
        self.image_Bowser=tk.PhotoImage(file='browser.png')

    #supprime un méga alien: appelé par manage_touched_mega_aliens_by de fleet
    def touched_by(self, canvas):
        canvas.delete(self.id)

    #sélectionne et créer l'image du Méga aliens: appelé dans install_in de fleet
    def install_in(self, canvas, x, y,cpt):
        if x==100:
            img=self.image_King           
        elif x==287:
            img=self.image_Crocodile
        else:
            img=self.image_Bowser
        self.id=canvas.create_image(x,y,image=img)

In [8]:
class SpaceInvaders():
    def __init__(self): 
        self.root = tk.Tk()
        self.root.title("Space Invaders")
        self.frame = tk.Frame(self.root)
        self.frame.pack(side="top", fill="both")
        self.game = Game(self.frame)
    def play(self):
        self.game
        self.root.bind("<Key>", self.game.keypress)
        self.root.mainloop()
                     
SpaceInvaders().play()